In [1]:
# Greedy FTL implementation
# For every  Write(LBA): 
# Check if there is a current open block. If not, open one from free list. If free-list has 2 or fewer blocks, goto GC first and then retry 1.
# Lookup physical address for the LBA in L2P
# Determine block 
# Decrement valid counter of the block
# Determine write pointer of current open block
# Compute new P_addr = block + write_pointer
# Update L2P by overwriting old P_addr with new P_addr for the LBA

import csv
import sys
import pandas as pd
import os
import glob
import itertools
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import random
import copy





In [2]:
lba_list = []

path = r'C:\Users\cchak\Desktop\Data_ECML\MSR_Traces\msr_2\converted'
all_files = glob.glob(os.path.join(path, "mds_*"))
print(all_files)

['C:\\Users\\cchak\\Desktop\\Data_ECML\\MSR_Traces\\msr_2\\converted\\mds_0.csv_output']


In [3]:
f = all_files[0]  # Change the file name as required


print("Working with file " + str(f))
cols = ['IO_num','Timestamp','LBA','Deathtime_RWI','Size']
df = pd.read_csv(f,engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False)
df.columns = cols
print(len(df))

Working with file C:\Users\cchak\Desktop\Data_ECML\MSR_Traces\msr_2\converted\mds_0.csv_output
1035178


In [4]:
lba_list = df['LBA'].tolist()
size_list = df['Size'].tolist()

lba_list =lba_list
size_list =size_list
print(len(lba_list))
print(len(size_list))

1035178
1035178


In [5]:
df.head(10)

,IO_num,Timestamp,LBA,Deathtime_RWI,Size
0,0,128166372011594132,3154132992,13,4096
1,1,128166372011600556,3201662976,256271,20480
2,2,128166372011606862,3154124800,55,4096
3,3,128166372017062367,57819136,440,4096
4,4,128166372017074528,57823232,440,4096
5,5,128166372017085606,57806848,440,4096
6,6,128166372017220587,3154137088,12,4096
7,7,128166372017230760,57810944,440,4096
8,8,128166372020823372,156237824,220,4096
9,9,128166372052829342,3201683456,113544,4096


In [ ]:
# Changing some size to test

size_list[0] = 300
size_list[1] = 400
size_list[2] = 5000
size_list[3] = 300
size_list[4] = 400



In [ ]:
page_size = 256
block_size = 4 * page_size
TB = 1000000000000
GB = 1000000000
page_per_block = 4
over_provisioning_ratio = 0.3


ssd_capacity = (0.05 * GB) 
start_OP_capacity = ssd_capacity  + page_size
end_OP_capacity = start_OP_capacity + (ssd_capacity*over_provisioning_ratio) 


# Make the block,page and physical addresses for normal and Overprovisioned capacity
page_addresses = []
block_addresses = []
page_addresses_OP = []
block_addresses_OP = []

block_placement = 0
start_counter = 0

while(start_counter < end_OP_capacity):
    if(start_counter < ssd_capacity - (page_per_block*page_size)):
        page_addresses.append(int(start_counter))
  
        if(block_placement >= page_per_block or start_counter == 0):
            block_addresses.append(int(start_counter))
            block_placement = 0

    else:
        page_addresses_OP.append(start_counter)
        if(block_placement == page_per_block):
            block_addresses_OP.append(int(start_counter))
            block_placement = 0
  

    block_placement = block_placement + 1
    start_counter = start_counter + page_size


print(len(page_addresses))
print(len(block_addresses))
print(len(page_addresses_OP))
print(len(block_addresses_OP))





In [ ]:
# Make L2P, free list and closed list
# Make Block-Struct
# Exists for each block in the system and holds the following variables: 
# Invalid-count: Number of invalid pages in the block
# Valid-count: Specifies number of valid pages in a block
# Write-pointer: Current Location of the write pointer

block_info ={}

for x in block_addresses:
    valid_count = page_per_block
    write_pointer = 0
    start_position = x
    block_info[x] = [start_position,write_pointer,valid_count]

print(len(block_info))

block_info_OP ={}

for x in block_addresses_OP:
    valid_count = page_per_block
    write_pointer = 0
    start_position = x
    block_info_OP[x] = [start_position,write_pointer,valid_count]

print(len(block_info_OP))



In [ ]:
# Initializing variables

page_valid = copy.deepcopy(page_addresses)
page_invalid = []


closed_blocks_physical_OP = []
free_blocks_physical_OP = copy.deepcopy(block_addresses_OP)
open_blocks_physical_OP = []

closed_blocks_physical = []
free_blocks_physical = copy.deepcopy(block_addresses)
open_blocks_physical = []

In [ ]:
# Initialzing pointers

counter = -1
block_pointer = int(min(block_addresses)) # Pointer to physical memory addresses in normal space
block_pointer_OP = int(min(block_addresses_OP)) # Pointer to physical memory addresses in Overprovisioned space
error_blocks = []

L2S = {}
L2P = {}
L2S_OP = {}
L2P_OP = {}


blocks_reclaimed = 0

# Cycling through IO Accesses
while(counter < len(lba_list) - 1):
    
    if(counter == len(lba_list) - 2 ):
        print("End of Trace!!")
    
    # If block pointer reaches end of the ssd_capacity, reset pointer 
    if(block_pointer > ssd_capacity - (page_per_block*page_size)):
        print("Block Pointer Reset")
        block_pointer = int(min(block_addresses))
        open_blocks_physical = list(set(open_blocks_physical))
        free_blocks_physical = list(set(free_blocks_physical))

    # Edge case : In case we run out of overprovisioned space
    if(block_pointer_OP >= end_OP_capacity):
        print("Something strange happened!!Out of overprovisioned blocks!!")
        block_pointer_OP = int(min(block_addresses_OP))

    
    counter = counter  + 1
    if(counter > 1000 and counter%1000 == 0):
        print("Percentage Done :" + str(counter/len(lba_list)*100)) # To check progress every 1000 IO
        
        
    lba  = int(lba_list[counter])
    size = int(size_list[counter])
    
    

    # When a page is visited again, invalidate the prior pages
    if(lba in L2P):
        if (lba not in L2S):
            print("Something strange just happened!!")
            print(str(lba)+" present in L2P, but not L2S")
            print("In other words, we don't know the size of the previous LBA access")
        else:
            size_prev = L2S[lba]
#         Invalidate physical pages while size is not 0 ; Remove from L2P ; Add to page_invalid
            physical_add = L2P[lba]
            while(size_prev > 0):
                if(lba in L2P):
                    L2P.pop(lba)
                if(lba in L2S):
                    L2S.pop(lba)
                if(physical_add not in page_invalid):
                    page_invalid.append(physical_add)
                size_prev = size_prev - page_size
                physical_add = physical_add + page_size

 
 
  # Assign data to open block
    elif((len(open_blocks_physical)*page_size) > (size*2)):

        # Assign L2S
        if(size > 0):
            if lba not in L2S:
                L2S[lba]= size
        
        # Adding to open list of not present
        open_blocks_physical = list(set(open_blocks_physical))    
        if(block_pick in free_blocks_physical):
            free_blocks_physical.remove(block_pick)
        if(block_pick not in open_blocks_physical):
            open_blocks_physical.append(block_pick)
            
        # Cleaning free , open and closed block data (Validation Check : Can be skipped)
        for x in free_blocks_physical:
            tmp = block_info[x]
            if (tmp[2] !=4):
                if(x in free_blocks_physical):
                    free_blocks_physical.remove(x)
            elif(tmp[2] > 0):
                if(x not in open_blocks_physical):
                    open_blocks_physical.append(x)
            elif(tmp[2] <= 0):
                if(x not in closed_blocks_physical):
                    open_blocks_physical.append(x)
        
            
        # Assign open block to the data until data is finished. Update L2P.
        while(size > 0 and ((len(open_blocks_physical)*page_size) > size)):
            # Find open block from the list of open blocks (Choose the minimum address one)
            flag =0
            while(flag !=1):
                block_pick = min(open_blocks_physical)
                tmp = block_info[block_pick]
                if(tmp[2] <=0):
                    if(block_pick in free_blocks_physical):
                        free_blocks_physical.remove(block_pick)
                    if(block_pick in open_blocks_physical):
                        open_blocks_physical.remove(block_pick)
                    if(block_pick not in closed_blocks_physical):
                        closed_blocks_physical.append(block_pick)
                    block_pick = min(open_blocks_physical)
                    tmp = block_info[block_pick]
                    flag=1
                flag =1
            
            valid_count = block_info[block_pick][2]
            write_pointer = block_info[block_pick][1]
            # Assigning data until size is 0
            while((valid_count > 0) and (size > 0) and (len(open_blocks_physical)*page_size > size)):
                # Assign L2P

                start_physical = block_pick + (write_pointer*page_size)
                L2P[lba]= start_physical
                L2S[lba]= size
                # Update block_info 
                tmp = block_info[block_pick]
                # Decrease invalid_count and update write_pointer
                tmp[2] = tmp[2] - 1
                tmp[1] = tmp[1] + 1
                block_info[block_pick]  = tmp
                
                valid_count = tmp[2]
                write_pointer = tmp[1]
                lba = lba + page_size
                size = size - page_size
#                 print(L2P)
                # If no more free pages in the block, add to closed_block_list
                if(valid_count <= 0):
                    if(block_pick in open_blocks_physical):
                        open_blocks_physical.remove(block_pick)
                    if(block_pick not in closed_blocks_physical):
                        closed_blocks_physical.append(block_pick)
                    


   
    # Assign data to free block
    elif((len(free_blocks_physical)*(page_per_block*page_size)) > (size*2)):
        if(size > 0):
            if lba not in L2S:
                L2S[lba]= size
                
        # Validation 
        free_blocks_physical = list(set(free_blocks_physical))    
        if(block_pointer > ssd_capacity - (page_per_block*page_size)):
            print("Block Pointer Reset")
            block_pointer = int(min(block_addresses))
            open_blocks_physical = list(set(open_blocks_physical))
            free_blocks_physical = list(set(free_blocks_physical))
        
        # Cleaning free , open and closed block data
        for x in free_blocks_physical:
            tmp = block_info[x]
            if (tmp[2] !=4):
                if(x in free_blocks_physical):
                    free_blocks_physical.remove(x)
            elif(tmp[2] > 0):
                if(x not in open_blocks_physical):
                    open_blocks_physical.append(x)
            elif(tmp[2] <= 0):
                if(x not in closed_blocks_physical):
                    open_blocks_physical.append(x)
        
        # Finding a free block in physical_free and assign logical block to it
        
        if(block_pointer not in free_blocks_physical):
             while(block_pointer not in free_blocks_physical):
                block_pointer = block_pointer + (page_per_block*page_size)
                if(block_pointer > ssd_capacity - (page_per_block*page_size)):
                    print("Block Pointer Reset")
                    block_pointer = int(min(block_addresses))
                    open_blocks_physical = list(set(open_blocks_physical))
                    free_blocks_physical = list(set(free_blocks_physical))
                    tmp = block_info[block_pointer]
                    if (tmp[2] !=4):
                        if(x in free_blocks_physical):
                            free_blocks_physical.remove(x)
            
            
        block_pick = block_pointer
          # Update open/closed/free block data (Once)
        if(block_pick in free_blocks_physical):
            free_blocks_physical.remove(block_pick)
            if(block_pick not in open_blocks_physical):
                open_blocks_physical.append(block_pick)

        valid_count = block_info[block_pick][2]
        write_pointer = block_info[block_pick][1]

        # Assigning data
        while(valid_count > 0 and size > 0 and (len(free_blocks_physical)*(page_per_block*page_size) > size)):
             # Assign L2P
            start_physical = block_pick + (write_pointer*page_size)
            L2P[lba]= start_physical
            L2S[lba]= size
            # Update block_info: Decrease valid_count and update write_pointer
            tmp = block_info[block_pick]
            tmp[2] = tmp[2] - 1
            tmp[1] = tmp[1] + 1
            block_info[block_pick]  = tmp

            write_pointer = tmp[1]
            valid_count = tmp[2]

            lba = lba + page_size
            size = size - page_size

            # If no more free pages in the block, add to closed_block_list
            if(valid_count <= 0):
                if(block_pick in open_blocks_physical):
                    open_blocks_physical.remove(block_pick)
                if(block_pick not in closed_blocks_physical):
                    closed_blocks_physical.append(block_pick)
                block_pointer = block_pointer + (page_per_block*page_size)
                if(block_pointer > ssd_capacity - (page_per_block*page_size)):
                    print("Block Pointer Reset")
                    block_pointer = int(min(block_addresses))
                    open_blocks_physical = list(set(open_blocks_physical))
                    free_blocks_physical = list(set(free_blocks_physical))

                
    # Need to perform GC, No free or open blocks available (adjust numbers to force GC earlier)
    else:
        print("Counter " +str(counter))
        print("Out of free and open blocks... Triggering Garbage Collection!!")
        print("Number of invalid pages to recycle " + str(len(page_invalid)))
        print("Number of free blocks before GC: " + str(len(free_blocks_physical)))
        
        
        # Add remaining size to page_invalid
#         For all invalid pages, find the list of victim blocks
        block_pointer_OP = int(min(block_addresses_OP))
        page_invalid = list(set(page_invalid))
        victim_block_data = {}
        for page in page_invalid:
            block_start =  int(page - (page%(page_size*page_per_block)))
            page_position = int((page - block_start)/page_size)
            if (block_start not in block_addresses):
                print("Something bad happened!! Cannot find block associated with the page")
                error_blocks.append(block_start)
            else:
                if(block_start not in victim_block_data):
                    tmp = []
                    tmp.append(page_position)
                    victim_block_data[block_start] = tmp
                else:
                    holder = victim_block_data[block_start]
                    holder.append(page_position)
                    victim_block_data[block_start] = holder
                    
        print("Need to claim " + str(len(page_invalid)) + " pages in " + str(len(victim_block_data))  + " blocks")


        # For each victim block, Copy invalid data to OP, add the block to free list
        for block in victim_block_data:
            page_pos_save = victim_block_data[block]
            # if len(page_pos_save) = 4, then all pages invalid, just add the block to free
            if(len(page_pos_save) == page_per_block):
                if (block in closed_blocks_physical):
                    closed_blocks_physical.remove(block)
                block_info[block] = [block,0,page_per_block]
                free_blocks_physical.append(block)
            else:
                # if len(page_pos_save) < 4, then find valid pages and copy them to OP, add block to free
                tmp = block_info[block]
                write_counter = -1
                while(write_counter < page_per_block and len(page_pos_save) != page_per_block):
                    write_counter = write_counter + 1
                    if (write_counter not in page_pos_save):
                        logical_address = block + (write_counter*page_per_block)
                        tmp_OP = block_info_OP[block_pointer_OP]
                        if(tmp_OP[1] == (page_per_block-1)):
                            block_pointer_OP = block_pointer_OP + (page_per_block*page_size)
                        # Update block info (Decrease valid_count and update write_pointer)
                        tmp[2] = tmp[2] + 1
                        page_pos_save.append(write_counter)
                        
                        # Update block info (Decrease valid_count and update write_pointer) for OP
                        tmp_OP[2] = tmp_OP[2] - 1
                        tmp_OP[1] = tmp_OP[1] + 1
                        block_info_OP[block_pointer_OP] = tmp_OP
                        
                        valid_count_OP = tmp_OP[1]
                        
                        if(valid_count_OP >= (page_per_block-1)):
                            block_pointer_OP = block_pointer_OP + (page_per_block*page_size)
                        physical_address = tmp_OP[0] + tmp_OP[1]*page_size
                        
                        L2P_OP[physical_address] = logical_address
                        blocks_reclaimed = blocks_reclaimed + 1
                        if(write_counter == page_per_block):
                            if (block in closed_blocks_physical):
                                closed_blocks_physical.remove(block)
                            if (block in open_blocks_physical):
                                open_blocks_physical.remove(block)
                            free_blocks_physical.append(block)             
                            block_info[block] = [block,0,page_per_block]
                            
        print("GC writes " + str(blocks_reclaimed))
       
        # Copy data from OP capacity to free blocks now
        start_block_pointer_OP = int(min(block_addresses_OP))
        current_block_pointer_OP = block_pointer_OP
        # Check if there are enough free blocks to store the data in normal  capacityy
            
        if((current_block_pointer_OP - start_block_pointer_OP) >= (len(free_blocks_physical)*page_per_block*page_size)):
            print("Not enough free blocks to copy overprovisioned data")
            
        if(block_pointer >= max(block_addresses)):
            block_pointer = int(min(block_addresses))
        
        print("Number of free blocks before TRIM " + str(len(free_blocks_physical)))
        for x in free_blocks_physical:
            tmp = block_info[x]
            if (tmp[2] !=4):
                if(x in free_blocks_physical):
                    free_blocks_physical.remove(x)
            elif(tmp[2] > 0):
                if(x not in open_blocks_physical):
                    open_blocks_physical.append(x)
            elif(tmp[2] <= 0):
                if(x not in closed_blocks_physical):
                    open_blocks_physical.append(x)
            
        print("Number of free blocks after TRIM: " + str(len(free_blocks_physical))) 
        
        while(start_block_pointer_OP <=current_block_pointer_OP):
        # Find the next free block
            if(block_pointer not in free_blocks_physical):
                 while(block_pointer not in free_blocks_physical):
                        block_pointer = block_pointer + (page_per_block*page_size)
                        if(block_pointer > ssd_capacity - page_size):
                            print("Block Pointer Reset")
                            block_pointer = int(min(block_addresses))
            block_pick = block_pointer 
            
            if(block_pick in free_blocks_physical):
                free_blocks_physical.remove(block_pick)
                if(block_pick not in open_blocks_physical):
                    open_blocks_physical.append(block_pick)
                
            valid_count = block_info[block_pick][2]
            write_pointer = block_info[block_pick][1]
            
            valid_count_OP = block_info_OP[start_block_pointer_OP][2]
            write_pointer_OP = block_info_OP[start_block_pointer_OP][1]
            
            while(valid_count > 0):
                 # Assign L2P
                start_logical = block_pick + (write_pointer*page_size)
                start_physical = start_block_pointer_OP + (write_pointer_OP*page_size)
                L2P[start_logical]= start_physical
                L2S[start_logical]= page_size
                # Update block_info 
                tmp = block_info[block_pick]
                tmp_OP = block_info_OP[start_block_pointer_OP]
                # Decrease valid_count and update write_pointer
                tmp[2] = tmp[2] - 1
                tmp[1] = tmp[1] + 1
                
                tmp_OP[2] = tmp_OP[2] - 1
                tmp_OP[1] = tmp_OP[1] + 1
                
                block_info[block_pick]  = tmp
                write_pointer = tmp[1]
                valid_count = tmp[2]
                
                block_info_OP[start_block_pointer_OP]  = tmp_OP
                write_pointer_OP = tmp[1]
                valid_count_OP = tmp[2]
                
                # If no more free pages in the block, add to closed_block_list
                if(valid_count <= 0):
                    if block_pick in open_blocks_physical:
                        open_blocks_physical.remove(block_pick)
                    
                    if(block_pointer >= max(block_addresses)):
                        block_pointer = min(block_addresses)
                    else:
                        block_pointer = block_pointer + (page_per_block*page_size)
                    if(block_pointer not in closed_blocks_physical):
                        closed_blocks_physical.append(block_pick)
                if(valid_count_OP <= 0):
                    start_block_pointer_OP = start_block_pointer_OP + (page_per_block*page_size)
     
        print("Number of free blocks after GC: " + str(len(free_blocks_physical)))
        print("Number of blocks reclaimed: " + str(blocks_reclaimed))
                   

#     else:
#         print(size)
#         print(counter)
#         print(len(free_blocks_physical))
#         print(len(open_blocks_physical))
#         print("Something bad happened!! No free or Open blocks found and still GC not triggered!!")
#         break
              
        

                